In [ ]:
load_model = False              # False    #True
KAGGLE = True                  #True   #False


print("load_model = ", load_model)
print("KAGGLE = ", KAGGLE)

In [ ]:
if KAGGLE:
    # This Python 3 environment comes with many helpful analytics libraries installed
    # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
    # For example, here's several helpful packages to load

    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

    # Input data files are available in the read-only "../input/" directory
    # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

    import os
    
    
    
#     > ls /kaggle
#     input/  lib/  working/
#     > pwd
#     '/kaggle/working'
#     > ls ../working
#     __notebook_source__.ipynb



#     ../input/petfinder-pawpularity-score/sample_submission.csv
#     ../input/mypetmodels/mymodel_epoch8.pth
#     ../input/pythonbox/Box-master/.github
#     ../input/timmpackage/pytorch-image-models-master/
#     ../working



#     for dirname, _, filenames in os.walk('/kaggle/input'):
    for dirname, _, filenames in os.walk('/kaggle/working'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

    # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
    # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
if KAGGLE: 
    root_dir = '/kaggle/input/petfinder-pawpularity-score/'
#     model_dir = '/kaggle/input/trainedModels/'
    model_dir = '../input/mypetmodels'
else:
    root_dir = '../ori_data/petData/'
    model_dir = '.'

        
        
print("root_dir = ", root_dir)
print("model_dir = ", model_dir)

In [ ]:
# !pip install opencv-python
# !pip install python-box timm pytorch-lightning==1.4.0 grad-cam ttach

In [ ]:
# based on the post here: https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/275094

import sys


if KAGGLE:
#     sys.path.append("../input/Box-master/")
#     sys.path.append("../input/pytorch-image-models-master/")
    sys.path.append("../input/pythonbox/Box-master/")
    sys.path.append("../input/timmpackage/pytorch-image-models-master/")
else:
    packge_dir = '/home1/yhl/jupyter/petfinder/packages_dir/'
    sys.path.append(packge_dir + "Box-master/")
    sys.path.append(packge_dir + "pytorch-image-models-master/")

In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm
import gc

import torch
import torchvision.transforms as T
from torchvision.io import read_image

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
# from pytorch_grad_cam import GradCAMPlusPlus
# from pytorch_grad_cam.utils.image import show_cam_on_image


import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

from pathlib import Path
import time


warnings.filterwarnings("ignore")

# Config

In [ ]:
batch_size_train_val = 32   #32    #31
#32
#96   #64

print("batch_size_train_val = ", batch_size_train_val)

In [ ]:
config = {'seed': 2021,
#           'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'root': root_dir, 
          'model_dir': model_dir,
          'n_splits': 5,
          'epoch': 20,
          'trainer': {
              'gpus': 1,
              'accumulate_grad_batches': 1,
              'progress_bar_refresh_rate': 1,
              'fast_dev_run': False,
              'num_sanity_val_steps': 0,
              'resume_from_checkpoint': None,
          },
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'train_loader':{
              'batch_size': batch_size_train_val,     #64,
              'shuffle': True,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': batch_size_train_val,     #64,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'model':{
#               'name': Path(model_dir) / 'swin_tiny_patch4_window7_224',
              'name': 'swin_large_patch4_window7_224',     
              #'resnet34', #'swin_large_patch4_window7_224', #'swin_tiny_patch4_window7_224',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config = Box(config)

In [ ]:
print(config)

In [ ]:
# print(config.optimizer.params.lr)
# # 1e-05

# print(config.seed)
# # 2021

In [ ]:
# print(config.seed)
# # 2021

torch.autograd.set_detect_anomaly(True)
seed_everything(config.seed)

# Read data

In [ ]:
train_df = pd.read_csv(os.path.join(config.root, "train.csv"))
test_df = pd.read_csv(os.path.join(config.root, "test.csv"))

In [ ]:
print(train_df.shape)
print(train_df.columns.values)
print(train_df.head(2))

print("\n")
print(test_df.shape)
print(test_df.columns.values)
print(test_df.head(2))

In [ ]:
train_df["imgPath"] = train_df["Id"].apply(lambda x: os.path.join(config.root, "train", x + ".jpg"))
test_df["imgPath"] = test_df["Id"].apply(lambda x: os.path.join(config.root, "test", x + ".jpg"))

In [ ]:
print(train_df.shape)
print(train_df.columns.values)
print(train_df.head(2))

print("\n")
print(test_df.shape)
print(test_df.columns.values)
print(test_df.head(2))

In [ ]:
rowsNum = train_df.shape[0]
assert(rowsNum == 9912)
print("rowsNum = ", rowsNum)


valid_pct=0.2
seed=999
print("valid_pct = ", valid_pct)
print("seed = ", seed)
np.random.seed(seed)
a = (np.random.rand(rowsNum) < valid_pct).astype(bool)


# print(a.shape[0])
assert(a.shape[0] == rowsNum)
valRowNum = np.sum(a)
realValPerc = np.sum(a)/rowsNum
print("valRowNum = ", valRowNum)
print("realValPerc = ", realValPerc)


train_df['valid_col'] = a
first20_validCol = train_df['valid_col'].values[:20].astype('int') 
target20 = np.array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])
print( first20_validCol )
assert( (first20_validCol == target20).all() )

In [ ]:
print(train_df.shape)
print(train_df.columns.values)
print(train_df.head(2))

print("\n")
print(test_df.shape)
print(test_df.columns.values)
print(test_df.head(2))

In [ ]:
train_df_train = train_df[train_df['valid_col'] == False]
train_df_val = train_df[train_df['valid_col'] == True]

print('train_df.shape = ', train_df.shape)
print("train_df_train.shape = ", train_df_train.shape)
print("train_df_val.shape = ", train_df_val.shape)
print('test_df.shape = ', test_df.shape)
assert(train_df.shape[0] == train_df_train.shape[0] + train_df_val.shape[0]) 
assert(train_df_val.shape[0] == valRowNum) 

# Dataset

In [ ]:
# # print(config.seed)
# # # 2021

# torch.autograd.set_detect_anomaly(True)
# seed_everything(config.seed)

In [ ]:
# seed_everything??

# # random.seed(seed)
# # np.random.seed(seed)
# # torch.manual_seed(seed)
# # torch.cuda.manual_seed_all(seed)

In [ ]:
# class PetfinderDataModule(LightningDataModule):
#     def __init__(
#         self,
#         train_df,
#         val_df,
#         cfg,
#     ):
#         super().__init__()
#         self._train_df = train_df
#         self._val_df = val_df
#         self._cfg = cfg

#     def __create_dataset(self, train=True):
#         return (
#             PetfinderDataset(self._train_df, self._cfg.transform.image_size)
#             if train
#             else PetfinderDataset(self._val_df, self._cfg.transform.image_size)
#         )

#     def train_dataloader(self):
#         dataset = self.__create_dataset(True)
#         return DataLoader(dataset, **self._cfg.train_loader)

#     def val_dataloader(self):
#         dataset = self.__create_dataset(False)
#         return DataLoader(dataset, **self._cfg.val_loader)

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, df, image_size=224):
#         self._X = df["Id"].values
        self._X = df["imgPath"].values
    
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
            
        self._transform = T.Resize([image_size, image_size])
        
        dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
        ]
        self._dense_features = df[dense_features].values

        
    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        
        features = self._dense_features[idx, :]
        
        if self._y is not None:
            label = self._y[idx]
            return image, label
        else:
            return image

In [ ]:
class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        df,
        is_valid,
        cfg,
    ):
        super().__init__()
        self._df = df
        self._is_valid = is_valid
        self._cfg = cfg

    def __create_dataset(self):
        return PetfinderDataset(self._df, self._cfg.transform.image_size)

    def get_dataloader(self):
        dataset = self.__create_dataset()
        if self._is_valid==False:
            return DataLoader(dataset, **self._cfg.train_loader)
        else:
            return DataLoader(dataset, **self._cfg.val_loader)        

In [ ]:
print(config.train_loader)
print(config.val_loader)

In [ ]:
# 'train_loader': {'batch_size': 64,
#                   'drop_last': True,
#                   'num_workers': 4,
#                   'pin_memory': False,
#                   'shuffle': True},
    
# 'val_loader': {'batch_size': 64,
#                 'drop_last': False,
#                 'num_workers': 4,
#                 'pin_memory': False,
#                 'shuffle': False}}

In [ ]:
# sample_dataloader = PetfinderDataModule(df, df, config).val_dataloader()
is_valid = True
test_dataloader = PetfinderDataModule(test_df, is_valid, config).get_dataloader()
test_dataloader_bNum = len(test_dataloader)
print("test_dataloader_bNum = ", test_dataloader_bNum)



# sample_dataloader = PetfinderDataModule(df, df, config).val_dataloader()
is_valid = True
val_dataloader = PetfinderDataModule(train_df_val, is_valid, config).get_dataloader()
val_dataloader_bNum = len(val_dataloader)
print("val_dataloader_bNum = ", val_dataloader_bNum)


# sample_dataloader = PetfinderDataModule(df, df, config).val_dataloader()
is_valid = False
train_dataloader = PetfinderDataModule(train_df_train, is_valid, config).get_dataloader()
train_dataloader_bNum = len(train_dataloader)
print("train_dataloader_bNum = ", train_dataloader_bNum)


# test_dataloader_bNum =  1
# val_dataloader_bNum =  64
# train_dataloader_bNum =  246

In [ ]:
# train_df.shape =  (9912, 16)
# train_df_train.shape =  (7886, 16)
# train_df_val.shape =  (2026, 16)
# test_df.shape =  (8, 14)

print('train_df.shape = ', train_df.shape)
print("train_df_train.shape = ", train_df_train.shape)
print("train_df_val.shape = ", train_df_val.shape)
print('test_df.shape = ', test_df.shape)

print("\n")
print("train_dataloader_bNum = ", train_dataloader_bNum)
print("val_dataloader_bNum = ", val_dataloader_bNum)
print("test_dataloader_bNum = ", test_dataloader_bNum)


print("\n")
print("batch_size_train_val = ", batch_size_train_val)
# assert(config.train_loader.batch_size == 64)
# assert(config.val_loader.batch_size == 64)
assert(config.train_loader.batch_size == batch_size_train_val)
assert(config.val_loader.batch_size == batch_size_train_val)
print("train_dataloader_bNum*batch_size_train_val = ", train_dataloader_bNum*batch_size_train_val)
print("val_dataloader_bNum*batch_size_train_val = ", val_dataloader_bNum*batch_size_train_val)
print("(train_dataloader_bNum*batch_size_train_val-7886) = ", (train_dataloader_bNum*batch_size_train_val-7886) )
print("(val_dataloader_bNum*batch_size_train_val-2026) = ", (val_dataloader_bNum*batch_size_train_val-2026) )
train_val_totalsum = (train_dataloader_bNum*batch_size_train_val + val_dataloader_bNum*batch_size_train_val)
print("(train_val_totalsum-9912) = ", (train_val_totalsum-9912) )
print("train_val_totalsum = ", train_val_totalsum)


# print("\n")
# # print("test_dataloader_bNum*batch_size_train_val = ", test_dataloader_bNum*batch_size_train_val)
# print("type(test_oneBatch) = ", type(test_oneBatch))
# print("test_oneBatch.shape = ", test_oneBatch.shape)



# train_df.shape =  (9912, 16)
# train_df_train.shape =  (7886, 16)
# train_df_val.shape =  (2026, 16)
# test_df.shape =  (8, 14)


# train_dataloader_bNum =  246
# val_dataloader_bNum =  64
# test_dataloader_bNum =  1


# batch_size_train_val =  32
# train_dataloader_bNum*batch_size_train_val =  7872
# val_dataloader_bNum*batch_size_train_val =  2048
# (train_dataloader_bNum*batch_size_train_val-7886) =  -14
# (val_dataloader_bNum*batch_size_train_val-2026) =  22
# (train_val_totalsum-9912) =  8
# train_val_totalsum =  9920

# Visualize data

In [ ]:
# oneBatch = iter(test_dataloader).next()
oneBatch = next(iter(test_dataloader))


isTest = True
print("isTest = ", isTest)
print("type(oneBatch) = ", type(oneBatch))


if isTest==False:
#     print(type(oneBatch))
    assert(type(oneBatch) == list)
#     assert(type(oneBatch) == tuple)
    oneBatchLen = len(oneBatch)
    assert(oneBatchLen == 2)
    print("len(oneBatch) = ", len(oneBatch))
    images, labels = oneBatch
else:
    assert(type(oneBatch) == torch.Tensor)
    print("oneBatch.shape = ", oneBatch.shape)
    images = oneBatch


plt.figure(figsize=(12, 12))
# for it, (image, label) in enumerate(zip(images[:16], labels[:16])):
for it, (image) in enumerate(images[:16]):
    plt.subplot(4, 4, it+1)
    plt.imshow(image.permute(1, 2, 0))
    plt.axis('off')
#     plt.title(f'Pawpularity: {int(label)}')

In [ ]:
# oneBatch = iter(val_dataloader).next()
oneBatch = next( iter(val_dataloader) )


isTest = False
print("isTest = ", isTest)
print("type(oneBatch) = ", type(oneBatch))


if isTest==False:
#     print(type(oneBatch))
    assert(type(oneBatch) == list)
#     assert(type(oneBatch) == tuple)
    oneBatchLen = len(oneBatch)
    assert(oneBatchLen == 2)
    images, labels = oneBatch
else:
    assert(type(oneBatch) == torch.Tensor)
    images = oneBatch


plt.figure(figsize=(12, 12))
if isTest:
    for it, (image) in enumerate(images[:16]):
        plt.subplot(4, 4, it+1)
        plt.imshow(image.permute(1, 2, 0))
        plt.axis('off')
else:
    for it, (image, label) in enumerate(zip(images[:16], labels[:16])):
    # for it, (image) in enumerate(images[:16]):
        plt.subplot(4, 4, it+1)
        plt.imshow(image.permute(1, 2, 0))
        plt.axis('off')
        plt.title(f'Pawpularity: {int(label)}')

In [ ]:
# https://stackoverflow.com/questions/21622193/python-3-2-coroutine-attributeerror-generator-object-has-no-attribute-next
# oneBatch = iter(train_dataloader).next()
oneBatch = next( iter(train_dataloader) )


isTest = False
print("isTest = ", isTest)
if isTest==False:
#     print(type(oneBatch))
    assert(type(oneBatch) == list)
#     assert(type(oneBatch) == tuple)
    oneBatchLen = len(oneBatch)
    assert(oneBatchLen == 2)
    images, labels = oneBatch
else:
    assert(type(oneBatch) == torch.Tensor)
    images = oneBatch


plt.figure(figsize=(12, 12))
if isTest:
    for it, (image) in enumerate(images[:16]):
        plt.subplot(4, 4, it+1)
        plt.imshow(image.permute(1, 2, 0))
        plt.axis('off')
else:
    for it, (image, label) in enumerate(zip(images[:16], labels[:16])):
    # for it, (image) in enumerate(images[:16]):
        plt.subplot(4, 4, it+1)
        plt.imshow(image.permute(1, 2, 0))
        plt.axis('off')
        plt.title(f'Pawpularity: {int(label)}')

# Augmentation

In [ ]:
# IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
# IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


# def get_default_transforms():
#     transform = {
#         "train": T.Compose(
#             [
#                 T.RandomHorizontalFlip(),
#                 T.RandomVerticalFlip(),
#                 T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
#                 T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
#                 T.ConvertImageDtype(torch.float),
#                 T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#             ]
#         ),
#         "val": T.Compose(
#             [
#                 T.ConvertImageDtype(torch.float),
#                 T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#             ]
#         ),
#     }
#     return transform

In [ ]:
# IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
# IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


# def get_default_transforms():
#     transform = {
#         "train": T.Compose(
#             [
# #                 T.RandomHorizontalFlip(),
# #                 T.RandomVerticalFlip(),
# #                 T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
# #                 T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
#                 T.ConvertImageDtype(torch.float),
# #                 T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#             ]
#         ),
#         "val": T.Compose(
#             [
#                 T.ConvertImageDtype(torch.float),
# #                 T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#             ]
#         ),
#     }
#     return transform

In [ ]:
# IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
# IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


# def get_default_transforms():
#     transform = {
#         "train": T.Compose(
#             [
# #                 T.RandomHorizontalFlip(),
# #                 T.RandomVerticalFlip(),
# #                 T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
# #                 T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
#                 T.ConvertImageDtype(torch.float),
#                 T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#             ]
#         ),
#         "val": T.Compose(
#             [
#                 T.ConvertImageDtype(torch.float),
#                 T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#             ]
#         ),
#     }
#     return transform

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
#                 T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform

# Model

In [ ]:
print("config.model.output_dim = ", config.model.output_dim)

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

In [ ]:
# class Model(pl.LightningModule):
#     def __init__(self, cfg):
#         super().__init__()
#         self.cfg = cfg
#         self.__build_model()
# #         self._criterion = eval(self.cfg.loss)()
# #         self.transform = get_default_transforms()
# #         self.save_hyperparameters(cfg)

#     def __build_model(self):
#         self.backbone = create_model(
#             self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
#         )
#         num_features = self.backbone.num_features
#         self.fc = nn.Sequential(
#             nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
#         )

#     def forward(self, x):
#         f = self.backbone(x)
#         out = self.fc(f)
#         return out

# Helper functions

In [ ]:
def delete_folder(myDir):
    import os
    import sys
    import shutil


#     # myfile="./swin_tiny_patch4_window7_224/"
#     myDir = f'{config.model_dir}/{config.model.name}'
    print("myDir = ", myDir)

    if os.path.isdir(myDir):
    # #     os.remove(myDir)
    #     os.rmdir(myDir)
        shutil.rmtree(myDir) 
        print("%s folder is removed" % myDir)
    else:    ## Show an error ##
        print("%s folder not found" % myDir)
        
    
    # importing os module
    import os

    try: 
        os.mkdir(myDir) 
        print("%s folder is created!" % myDir)
    except OSError as error: 
        print(error)  

In [ ]:
def cal_rmse(preds, labels):
     return torch.sqrt(((labels - preds) ** 2).mean()).item()

In [ ]:
def train_val_share_step(batch, mode, loss_fn):
    # mode: 'train', 'val'
    images, labels = batch
    labels = labels.float() / 100.0
    images = get_default_transforms()[mode](images)

#     if torch.rand(1)[0] < 0.5 and mode == 'train':
#         mix_images, target_a, target_b, lam = mixup(images, labels, alpha=0.5)
#         logits = self.forward(mix_images).squeeze(1)
#         loss = self._criterion(logits, target_a) * lam + \
#             (1 - lam) * self._criterion(logits, target_b)
#     else:
#         logits = self.forward(images).squeeze(1)
#         loss = self._criterion(logits, labels)

    logits = model.forward(images).squeeze(1)
    loss = loss_fn(logits, labels)
#     loss = loss.item()

    pred = logits.sigmoid().detach().cpu() * 100.
    labels = labels.detach().cpu() * 100.
    return loss, pred, labels

In [ ]:
# def train_val_share_step(batch, mode):
#     # mode: 'train', 'val'
#     images, labels = batch
#     labels = labels.float() / 100.0
#     images = get_default_transforms()[mode](images)

# #     if torch.rand(1)[0] < 0.5 and mode == 'train':
# #         mix_images, target_a, target_b, lam = mixup(images, labels, alpha=0.5)
# #         logits = self.forward(mix_images).squeeze(1)
# #         loss = self._criterion(logits, target_a) * lam + \
# #             (1 - lam) * self._criterion(logits, target_b)
# #     else:
# #         logits = self.forward(images).squeeze(1)
# #         loss = self._criterion(logits, labels)

#     logits = model.forward(images).squeeze(1)
#     loss = loss_fn(logits, labels)
# #     loss = loss.item()

#     pred = logits.sigmoid().detach().cpu() * 100.
#     labels = labels.detach().cpu() * 100.
#     return loss, pred, labels

In [ ]:
def get_min_minind(arr):
    return np.min(arr), np.argmin(arr)

In [ ]:
def pred_for_test_dataloader(model, test_dataloader):
    super_final_predictions = []

    with torch.no_grad():  # *2
        for org_images in test_dataloader:
            test_batch_size = len(org_images)
    # #         print("test_batch_size = ", test_batch_size)

    #         images = model.transform['val'](org_images)
            images = get_default_transforms()['val'](org_images)

            images = images.to(model.device)
            logits = model.forward(images).squeeze(1)
            pred = logits.sigmoid().detach().cpu().numpy() * 100

            super_final_predictions += list(pred)
            
    return super_final_predictions

In [ ]:
def train_myModel(train_dataloader, val_dataloader, model, loss_fn, optimizer, num_epoches, earlyStopPatience, bestModelName):
    train_loss_arr = []
    val_loss_arr = []
    val_rmse_arr = []
    best_val_rmse = 2000
    best_val_rmse_epoch = 0 

    
    # trainer.fit(model, train_dataloader, val_dataloader)
    for epoch in range(num_epoches):
        epoch_startTime = time.time()


        # train:
        model.train()  # *1
        train_loss = []
        for batch in tqdm(train_dataloader):
            batch = [x.to(device) for x in batch]

    #         x_train = x_train.to(device)
    #         y_train = y_train.to(device)

    #         y_hat = model(x_train)
    #         loss = loss_fn(y_train, y_hat)
    #         loss = loss.item()
    #         # The item() method extracts the loss’s value as a Python float. 

    
#             loss, pred, label = train_val_share_step(batch, 'train')
            loss, pred, label = train_val_share_step(batch, 'train', loss_fn)


            train_loss.append( loss.item() )

            loss.backward()        		# *4
            optimizer.step()        	# *5
            optimizer.zero_grad()        	# *6
        train_loss = torch.mean(torch.tensor(train_loss))
        train_loss = train_loss.item()
    #     print('train_loss: ', train_loss)


        # validation:
        model.eval()  # *3
        with torch.no_grad():  # *2
            val_loss = []
            preds = []
            labels = []
            for batch in tqdm(val_dataloader):
                batch = [x.to(device) for x in batch]
    #         for x_val, y_val in tqdm(val_dataloader):
    #             x_val = x_val.to(device)
    #             y_val = y_val.to(device)

    #             yhat = model(x_val)
    #             val_loss_val = loss_fn(y_val, yhat).item()

    
#                 loss, pred, label = train_val_share_step(batch, 'val')
                loss, pred, label = train_val_share_step(batch, 'val', loss_fn)


                val_loss.append( loss.item() )
                preds.append( pred )
                labels.append( label )
                
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        val_rmse = cal_rmse(preds, labels)
    #     print('val_loss: ', val_loss)
    

        val_loss = torch.mean(torch.tensor(val_loss))
        val_loss = val_loss.item()




        train_loss_arr.append(train_loss)
        val_loss_arr.append(val_loss)
        val_rmse_arr.append(val_rmse)

        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_val_rmse_epoch = epoch
            
#             bestModelName = 'mymodel_epoch' + str(best_val_rmse_epoch) + '.pth'
            torch.save(model, Path(myDir)/bestModelName)
    
            print(f"best model is saved.")
            print("bestModelName = ", bestModelName)
            print("best_val_rmse_epoch = ", best_val_rmse_epoch)
#             print(f"best model {bestModelName} is saved.")
            


        epoch_endTime = time.time()
        delta = epoch_endTime - epoch_startTime
        epochTimeInMin = delta/60

        print( f'epoch={epoch},  train_loss={train_loss},  val_loss={val_loss},  val_rmse={val_rmse}')
        print( f'best_val_rmse_epoch={best_val_rmse_epoch},  best_val_rmse={best_val_rmse},  epochTimeInMin={epochTimeInMin}')

        
    print("\n")
    print("all ecpoches finishes!")    
    res_df = pd.DataFrame()
    res_df['epoch'] = range(num_epoches)
    res_df['train_loss'] = train_loss_arr
    res_df['val_loss'] = val_loss_arr
    res_df['val_rmse'] = val_rmse_arr
    
    
    minValRmse, minValRmseInd = get_min_minind(res_df['val_rmse'])
    minValLoss, minValLossInd = get_min_minind(res_df['val_loss'])
    bestValOneRow = (best_val_rmse_epoch, res_df['epoch'][minValRmseInd], minValRmse, \
                                          res_df['epoch'][minValLossInd], minValLoss) 
    print(bestValOneRow)
    
#     assert(minValRmseInd == best_val_rmse_epoch)
#     assert(minValLossInd == minValLossInd)
    if not (minValRmseInd == best_val_rmse_epoch) or not(minValLossInd == best_val_rmse_epoch):
        print("Error: epochInd not equal!")
        print("best_val_rmse_epoch = ", best_val_rmse_epoch)
        print("minValRmseInd = ", minValRmseInd)
        print("minValLossInd = ", minValLossInd)
        print("res_df = ", res_df)
    
    
    return res_df, minValRmse, minValLoss, best_val_rmse_epoch

# Train

In [ ]:
print("load_model = ", load_model)

In [ ]:
if KAGGLE:
    # Making pretrained weights work without needing to find the default filename
    if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
        
#     ../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth
    !cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'
#     !cp '../input/petnets/swin_tiny_patch4_window7_224.pth' '/root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth'

In [ ]:
# Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth

In [ ]:
# if load_model==False:
#     if 'model' in locals():
#         del model
#     gc.collect()
#     model  = None
    

#     model = Model(config)

#     model = model.cuda()
#     device = model.device
#     print("device = ", device)

In [ ]:
if KAGGLE:
    if load_model==False:
        myDir = f'../working'
    else:
        myDir = f'{config.model_dir}'
else:
    print("config.model_dir = ", config.model_dir)
    print("config.model.name = ", config.model.name)
    myDir = f'{config.model_dir}/{config.model.name}'
    
print("type(myDir) = ", type(myDir) ) 
print("myDir = ", myDir)

In [ ]:
print("load_model = ", load_model)
print("KAGGLE = ", KAGGLE)

In [ ]:
if KAGGLE==False and load_model==False:
    delete_folder(myDir)

In [ ]:
# loss_fn = nn.MSELoss(reduction='mean')
# optimizer = optim.SGD(model.parameters(), lr=1e-1)

loss_fn = nn.BCEWithLogitsLoss()

num_epoches = 10
#10   #15  #20

earlyStopPatience = 3

In [ ]:
%%time

import gc


if load_model==False:
    model  = None
    model = Model(config)
    model = model.cuda()
    device = model.device
#     print("device = ", device)
    
    
    # optimizer = torch.optim.Adam(pytorch_model.parameters(), lr=1e-5)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    
#      # bestModelName =  'mymodel_epoch8.pth'
    bestModelName = 'mymodel_best.pth'
    
    train_res = train_myModel(train_dataloader, val_dataloader, model, loss_fn, optimizer, \
                                                   num_epoches, earlyStopPatience, bestModelName)
    res_df, minValRmse, minValLoss, best_val_rmse_epoch = train_res   
    
    
    print("\n")
    print("pred for test dataset!")    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    model  = None
    
    print("KAGGLE = ", KAGGLE)
    print("myDir = ", myDir)
    print("bestModelName = ", bestModelName)
    model = torch.load( Path(myDir)/bestModelName )
    model = model.cuda().eval()
    device = model.device
    print("device = ", device)
    
    super_final_predictions = pred_for_test_dataloader(model, test_dataloader)

    del model
    gc.collect()
    torch.cuda.empty_cache()
    

print("\n")

In [ ]:
if load_model==False:
    print("best_val_rmse_epoch = ", best_val_rmse_epoch)
    print("minValLoss = ", minValLoss)
    print("minValRmse = ", minValRmse)
    
#     print("minValRmseInd = ", minValRmseInd)
#     print("minValLossInd = ", minValLossInd)

    print(res_df)
    res_df[["train_loss", "val_loss"]].plot()
    res_df[["val_rmse"]].plot()

In [ ]:
# if load_model==False:
#     # import pandas as pd

#     # Calling DataFrame constructor
#     res_df = pd.DataFrame()

#     res_df['epoch'] = range(num_epoches)
#     res_df['train_loss'] = train_loss_arr
#     res_df['val_loss'] = val_loss_arr
#     res_df['val_rmse'] = val_rmse_arr
# #     res_df['val_rmse_norm'] = list( np.array(val_rmse_arr)/100.0 )

#     print(res_df)

In [ ]:
# if load_model==False:
# #     # res_df.plot()
# #     res_df[["train_loss", "val_loss", "val_rmse_norm"]].plot()
#     res_df[["train_loss", "val_loss"]].plot()
#     res_df[["val_rmse"]].plot()

In [ ]:
# print(res_df['val_rmse'])

# print("\n")
# print( np.min(val_rmse_arr) )
# ind = np.argmin(val_rmse_arr)
# print(ind)
# print(res_df['epoch'][ind])

In [ ]:
# print(res_df['val_loss'])

# print("\n")
# print( np.min(val_loss_arr) )
# ind = np.argmin(val_loss_arr)
# print(ind)
# print(res_df['epoch'][ind])

# Submission Csv

In [ ]:
# images = iter(test_dataloader).next()
images = next( iter(test_dataloader) )

plt.figure(figsize=(12, 12))
# for it, (image, label) in enumerate(zip(images[:16], labels[:16])):
for it, (image) in enumerate(images[:16]):
    test_img = image
    plt.subplot(4, 4, it+1)
    plt.imshow(image.permute(1, 2, 0))
    plt.axis('off')
#     plt.title(f'Pawpularity: {int(label)}')

In [ ]:
#     > ls /kaggle
#     input/  lib/  working/
#     > pwd
#     '/kaggle/working'
#     > ls ../working
#     __notebook_source__.ipynb



#     ../input/petfinder-pawpularity-score/sample_submission.csv
#     ../input/mypetmodels/mymodel_epoch8.pth
#     ../input/pythonbox/Box-master/.github
#     ../input/timmpackage/pytorch-image-models-master/
#     ../working

In [ ]:
# if KAGGLE:
#     if load_model==False:
#         myDir = f'../working'
#     else:
#         myDir = f'{config.model_dir}'
# else:
#     myDir = f'{config.model_dir}/{config.model.name}'

print("type(myDir) = ", type(myDir) ) 
print("myDir = ", myDir)
print("bestModelName = ", bestModelName)

In [ ]:
# import gc

# if 'model' in locals():
#     del model
# gc.collect()
# model  = None

In [ ]:
# from pathlib import Path

# # https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html

# # # model = torch.load('model.pth')
# # model = torch.load( Path(myDir)/ 'mymodel.pth' )


# # torch.save( model, Path(myDir)/bestModelName )
# model = torch.load( Path(myDir)/bestModelName )

In [ ]:
# model = model.cuda().eval()
# device = model.device
# print("device = ", device)

In [ ]:
# super_final_predictions = pred_for_test_dataloader(model, test_dataloader)

In [ ]:
print(len(super_final_predictions))
print(type(super_final_predictions))
# print(super_final_predictions)

# 8
# <class 'list'>

In [ ]:
# print("test_df.shape = ", test_df.shape)
# print("test_df.columns.values= ", test_df.columns.values )
# print("\n")
# print(test_df.head(5))


# print("\n")
# test_df["Pawpularity"] = super_final_predictions
# print("test_df.shape = ", test_df.shape)
# print("test_df.columns.values= ", test_df.columns.values )


# print("\n")
# test_df = test_df[["Id", "Pawpularity"]]
# print("test_df.shape = ", test_df.shape)
# print("test_df.columns.values= ", test_df.columns.values )
# test_df.to_csv("submission.csv", index=False)

In [ ]:
submit_test_df = pd.DataFrame()
submit_test_df["Id"] = test_df["Id"]
submit_test_df["Pawpularity"] = super_final_predictions

In [ ]:
print("submit_test_df.shape = ", submit_test_df.shape)
print("submit_test_df.columns.values= ", submit_test_df.columns.values )
submit_test_df.head()

In [ ]:
submit_test_df.to_csv("submission.csv", index=False)

In [ ]:
print("Done!")